In [ ]:
!pip install awswrangler


In [ ]:
import sys
from importlib import reload
import logging

reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

In [ ]:
import awswrangler as wr
import boto3

landing_bucket = "s3://tgedr-de-landing-dev/"
landing_key = "s3://tgedr-de-landing-dev/tickers"

In [ ]:
wr.s3.does_object_exist(landing_key)
df = wr.s3.read_parquet(landing_bucket)
df

In [ ]:
landing_output = "s3://tgedr-de-landing-dev/output"

wr.s3.to_parquet(
    df=df,
    path=landing_output,
    dataset=True,
    mode="overwrite",
    partition_cols=["variable"],
)

In [ ]:
databases = wr.catalog.databases()
print(databases)

In [ ]:
database = "tgedr_dataengineering"

if database not in databases.values:
    wr.catalog.create_database(database)
    print(wr.catalog.databases())
else:
    print("Database {database} already exists")

In [ ]:
wr.catalog.tables(database=database)

In [ ]:
res = wr.s3.store_parquet_metadata(
    path=landing_output,
    database="tgedr_dataengineering",
    table="just_a_test",
    dataset=True,
    mode="overwrite"
)

In [ ]:
for table in wr.catalog.get_tables(database=database):
    wr.catalog.delete_table_if_exists(database=database, table=table["Name"])

In [ ]:
wr.catalog.delete_database(database)